In [1]:
import pandas as pd
import numpy as np
import talib
from datetime import datetime
import os.path  # 管理路径

In [2]:
data = pd.read_excel('./复权后品种行情数据/棉花主连_2022-01-04_to_2023-09-05后复权数据.xlsx')

In [4]:
data

,时间,期货简称,期货代码,月合约代码,合约乘数,报价单位,最小变动价位,开盘价,最高价,最低价,...,持仓量,月合约代码_shift,前一日收盘价,切换日,复权系数_x,复权系数_y,复权开盘价,复权最高价,复权最低价,复权收盘价
0,2022-01-04,棉花主连,CFZL2,CF205,5,元（人民币）/吨,5,20690,20780,20590,...,435906,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-05,棉花主连,CFZL2,CF205,5,元（人民币）/吨,5,20740,21145,20700,...,439247,CF205,20670.0,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-06,棉花主连,CFZL2,CF205,5,元（人民币）/吨,5,21060,21105,20855,...,437177,CF205,21065.0,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-07,棉花主连,CFZL2,CF205,5,元（人民币）/吨,5,20950,21135,20905,...,442532,CF205,20900.0,False,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-10,棉花主连,CFZL2,CF205,5,元（人民币）/吨,5,21000,21195,20965,...,445534,CF205,21010.0,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,2023-08-30,棉花主连,CFZL2,CF401,5,元（人民币）/吨,5,17500,17630,17485,...,626480,CF401,17460.0,False,0.99104,1.038188,18168.292374,18303.256832,18152.719552,18282.493069
403,2023-08-31,棉花主连,CFZL2,CF401,5,元（人民币）/吨,5,17590,17700,17530,...,642207,CF401,17610.0,False,0.99104,1.038188,18261.729307,18375.930001,18199.438018,18334.402476
404,2023-09-01,棉花主连,CFZL2,CF401,5,元（人民币）/吨,5,17700,17905,17670,...,667264,CF401,17660.0,False,0.99104,1.038188,18375.930001,18588.758569,18344.784357,18469.366934
405,2023-09-04,棉花主连,CFZL2,CF401,5,元（人民币）/吨,5,17890,17900,17345,...,637956,CF401,17790.0,False,0.99104,1.038188,18573.185747,18583.567629,18007.373213,18085.237324


In [ ]:
import backtrader as bt  # 导入backtrader库

# 定义策略类
class MyStrategy(bt.Strategy):
    # 定义策略参数
    params = (
        ('shortlengh', 5),  # 短期均线周期
        ('longlengh', 30),  # 长期均线周期
        ('risk', 0.02),  # 风险系数
        ('backnumber', 3),  # 回溯周期
        ('dd', 4),  # 前高前低和今日差值
        ('ATR_n', 10),  # ATR周期
    )

    def __init__(self):
        # 初始化指标
        self.MA1 = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.shortlengh)  # 短期均线
        self.MA2 = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.longlengh)  # 长期均线
        self.ATR = bt.indicators.AverageTrueRange(self.data, period=self.params.ATR_n)  # ATR指标
        self.arr_high = []  # 阻力位列表
        self.arr_low = []  # 支撑位列表

    def next(self):
        # 计算交易单位
        size = self.broker.getcash() * self.params.risk / (self.ATR[0] * self.data._name)

        # 如果满足买入条件，则买入
        if len(self.arr_high) > 0 and self.MA1[0] > self.MA2[0] and self.data.high[0] >= self.arr_high[0] and self.data.high[-1] < self.arr_high[0]:
            self.buy(size=size)

        # 如果满足卖出条件，则卖出
        if len(self.arr_low) > 0 and self.MA1[0] < self.MA2[0] and self.data.low[0] <= self.arr_low[0] and self.data.low[-1] > self.arr_low[0]:
            self.sell(size=size)

        # 如果有持仓，检查是否需要止损
        if self.position:
            if self.position.size > 0 and self.data.low[0] <= self.highest[-1] - self.params.backnumber * self.ATR[0]:
                self.sell()

            if self.position.size < 0 and self.data.high[0] >= self.lowest[-1] + self.params.backnumber * self.ATR[0]:
                self.buy()

        # 如果数据足够，计算阻力位和支撑位
        if len(self) >= self.params.longlengh + self.params.dd:
            if self.data.high[-self.params.dd] == max(self.data.high.get(-self.params.dd-4, -self.params.dd+5)):
                self.arr_high.insert(0, self.data.high[-self.params.dd] * 0.99)

            if self.data.low[-self.params.dd] == min(self.data.low.get(-self.params.dd-4, -self.params.dd+5)):
                self.arr_low.insert(0, self.data.low[-self.params.dd] * 1.01)

# 创建Cerebro引擎
cerebro = bt.Cerebro()

# 添加数据
data = bt.feeds.YourDataFeed(dataname='your_data')
cerebro.adddata(data)

# 添加策略
cerebro.addstrategy(MyStrategy)

# 运行策略
cerebro.run()